# PIPE LINE

Xây dựng linear regression model để dự đoán thời lượng bay (flight duration) từ dow, org km (= mile * 1,60934)

## Đọc dữ liệu

In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
import findspark
findspark.init()

In [2]:
sc= SparkContext(appName= 'Demo linear regression', master= 'local')
ss= SparkSession(sc)

In [3]:
path= '/Users/vovanthuong/Desktop/9 - Big Data in Machine Learning/Data/Data ML/flights.csv'
df= ss.read.csv(path, inferSchema= True, header= True)

In [4]:
df.show(3)

+---+---+---+-------+------+---+----+------+--------+-----+
|mon|dom|dow|carrier|flight|org|mile|depart|duration|delay|
+---+---+---+-------+------+---+----+------+--------+-----+
| 11| 20|  6|     US|    19|JFK|2153|  9.48|     351|   NA|
|  0| 22|  2|     UA|  1107|ORD| 316| 16.33|      82|   30|
|  2| 20|  4|     UA|   226|SFO| 337|  6.17|      82|   -8|
+---+---+---+-------+------+---+----+------+--------+-----+
only showing top 3 rows



In [5]:
df.printSchema()

root
 |-- mon: integer (nullable = true)
 |-- dom: integer (nullable = true)
 |-- dow: integer (nullable = true)
 |-- carrier: string (nullable = true)
 |-- flight: integer (nullable = true)
 |-- org: string (nullable = true)
 |-- mile: integer (nullable = true)
 |-- depart: double (nullable = true)
 |-- duration: integer (nullable = true)
 |-- delay: string (nullable = true)



## Chia dữ liệu

In [6]:
train, test= df.randomSplit([0.8, 0.2])

## Xây dựng Pipeline

In [110]:
# Tạo đối tượng chuyển đổi cột  mile thành km
from pyspark.ml.feature import SQLTransformer
mile_km_converter= SQLTransformer(statement= 'SELECT dow, org, duration, round(mile * 1.60934) as km FROM __THIS__')

In [104]:
# Tạo các đối tượng mã hóa dữ liệu
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator
str_indexer= StringIndexer(inputCol= 'org', outputCol= 'org_index')
oh_encoder= OneHotEncoderEstimator(inputCols= ['org_index'], outputCols= ['org_dummy'])

In [105]:
# Tạo đối tượng để tạo vector
from pyspark.ml.feature import VectorAssembler
vector_assembler= VectorAssembler(inputCols= ['dow', 'org_dummy', 'km'], outputCol= 'features')

In [106]:
# Tạo đối tượng min-max scaler
from pyspark.ml.feature import MinMaxScaler
mm_scaler= MinMaxScaler(inputCol= 'features', outputCol= 'scale_features')

In [107]:
# Tạo đối tượng linear regression
from pyspark.ml.regression import LinearRegression
lir= LinearRegression(featuresCol= 'scale_features', labelCol= 'duration', predictionCol= 'duration_predict')

In [112]:
# Tạo pipeline
from pyspark.ml import Pipeline
pipeline_lir= Pipeline(stages= [mile_km_converter, str_indexer, oh_encoder, vector_assembler, mm_scaler, lir])

## Xây dựng mô hình dựa trên pipeline vừa tạo

In [113]:
lir_model= pipeline_lir.fit(train)

In [114]:
lir_model.transform(train).show(5)

+---+---+--------+----+---------+-------------+--------------------+--------------------+------------------+
|dow|org|duration|  km|org_index|    org_dummy|            features|      scale_features|  duration_predict|
+---+---+--------+----+---------+-------------+--------------------+--------------------+------------------+
|  2|JFK|     370|3983|      2.0|(7,[2],[1.0])|(9,[0,3,8],[2.0,1...|[0.33333333333333...|364.39507981006113|
|  2|JFK|     385|4162|      2.0|(7,[2],[1.0])|(9,[0,3,8],[2.0,1...|[0.33333333333333...| 377.6946656059827|
|  2|ORD|     560|6828|      0.0|(7,[0],[1.0])|(9,[0,1,8],[2.0,1...|[0.33333333333333...| 551.6582642722782|
|  2|SFO|     325|4352|      1.0|(7,[1],[1.0])|(9,[0,2,8],[2.0,1...|[0.33333333333333...|359.69161595492665|
|  2|JFK|     379|3983|      2.0|(7,[2],[1.0])|(9,[0,3,8],[2.0,1...|[0.33333333333333...|364.39507981006113|
+---+---+--------+----+---------+-------------+--------------------+--------------------+------------------+
only showing top 5 

## Đánh giá mô hình

metricName in evaluation - one of:
                       - rmse - root mean squared error (default)
                       - mse - mean squared error
                       - r2 - r^2 metric
                       - mae - mean absolute error

### Đánh giá trên tập dữ liệu train

In [117]:
train_result= lir_model.transform(train)

In [126]:
train_result.show(3)

+---+---+--------+----+---------+-------------+--------------------+--------------------+------------------+
|dow|org|duration|  km|org_index|    org_dummy|            features|      scale_features|  duration_predict|
+---+---+--------+----+---------+-------------+--------------------+--------------------+------------------+
|  2|JFK|     370|3983|      2.0|(7,[2],[1.0])|(9,[0,3,8],[2.0,1...|[0.33333333333333...|364.39507981006113|
|  2|JFK|     385|4162|      2.0|(7,[2],[1.0])|(9,[0,3,8],[2.0,1...|[0.33333333333333...| 377.6946656059827|
|  2|ORD|     560|6828|      0.0|(7,[0],[1.0])|(9,[0,1,8],[2.0,1...|[0.33333333333333...| 551.6582642722782|
+---+---+--------+----+---------+-------------+--------------------+--------------------+------------------+
only showing top 3 rows



In [121]:
from pyspark.ml.evaluation import RegressionEvaluator

124.61079304057594

In [123]:
for metric in ['rmse', 'mse', 'r2', 'mae']:
    value= RegressionEvaluator(labelCol= 'duration', predictionCol= 'duration_predict', metricName= metric).evaluate(train_result)
    print(metric, ':', value)

rmse : 11.16292045302554
mse : 124.61079304057594
r2 : 0.9835423247452423
mae : 8.52828055387588


### Đánh giá trên tập dữ liệu test

In [124]:
test_result= lir_model.transform(test)

In [127]:
test_result.show(3)

+---+---+--------+----+---------+-------------+--------------------+--------------------+------------------+
|dow|org|duration|  km|org_index|    org_dummy|            features|      scale_features|  duration_predict|
+---+---+--------+----+---------+-------------+--------------------+--------------------+------------------+
|  2|ORD|     135|1395|      0.0|(7,[0],[1.0])|(9,[0,1,8],[2.0,1...|[0.33333333333333...|147.98983058936142|
|  2|ORD|     130|1180|      0.0|(7,[0],[1.0])|(9,[0,1,8],[2.0,1...|[0.33333333333333...| 132.0154677618578|
|  2|ORD|     120|1180|      0.0|(7,[0],[1.0])|(9,[0,1,8],[2.0,1...|[0.33333333333333...| 132.0154677618578|
+---+---+--------+----+---------+-------------+--------------------+--------------------+------------------+
only showing top 3 rows



In [125]:
for metric in ['rmse', 'mse', 'r2', 'mae']:
    value= RegressionEvaluator(labelCol= 'duration', predictionCol= 'duration_predict', metricName= metric).evaluate(test_result)
    print(metric, ':', value)

rmse : 11.443495640246923
mse : 130.95359246835034
r2 : 0.9827628903787811
mae : 8.540627936051639
